- consider only item features
- modify functions for predictions of cold users
- pickle als model and user, item mapping
- remove cold prediction
- add autox

In [1]:
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

Cloning into 'autox'...
remote: Enumerating objects: 1785, done.
remote: Counting objects: 100% (1785/1785), done.
remote: Compressing objects: 100% (1212/1212), done.
remote: Total 1785 (delta 1173), reused 1133 (delta 528), pack-reused 0
Receiving objects: 100% (1785/1785), 8.03 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (1173/1173), done.
Processing ./autox
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 1.3 MB 598 kB/s            
  Created wheel for automl-x: filename=automl_x-0.2.1-py3-none-any.whl size=4648211 sha256=a9c371c8652f06b084cc86056d98634f0d4fe3cbc52144cbd3bdee7c0e7382f1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ovn9s36v/wheels/d8/d0/3f/17c06e69ec61f9c250a3274a013cc8a24b47965ec33f40c73d
Successfully built automl-x


In [2]:
!pip install similaripy

     |████████████████████████████████| 331 kB 595 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for similaripy: filename=similaripy-0.1.2-cp37-cp37m-linux_x86_64.whl size=2040902 sha256=44f996d70ca2ca19f53e90ee38ad21530e878bfd7a4bf414bfb3f0e2e5e52125
  Stored in directory: /root/.cache/pip/wheels/2e/81/4b/7a396ebc45534553ddee1eb106d99023df4d9d09b53b13cc66
Successfully built similaripy


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import implicit
import similaripy
import pickle
from autox.autox_competition.feature_engineer import fe_ima2vec

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

In [4]:
DATA_DIR = "/kaggle/input/h-and-m-personalized-fashion-recommendations/"
df = pd.read_csv(DATA_DIR + "transactions_train.csv")
article_df = pd.read_csv(DATA_DIR + "articles.csv")
customer_df = pd.read_csv(DATA_DIR + "customers.csv")
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [5]:
df["t_dat"] = pd.to_datetime(df["t_dat"])

date_week_df = df.drop_duplicates("t_dat")[["t_dat"]].reset_index(drop=True)
date_week_df['week_no'] = (date_week_df['t_dat'] + pd.DateOffset(days=5)).dt.week
date_week_df["week_no"] = date_week_df["week_no"].diff(1)
date_week_df["week_no"].fillna(0, inplace=True)
date_week_df["week_no"] = date_week_df["week_no"] != 0
date_week_df["week_no"] = date_week_df["week_no"].cumsum()

df = pd.merge(df, date_week_df, on="t_dat", how="left")

df.sort_values(['t_dat', 'customer_id'], inplace=True)

test_week = df.week_no.max()
last_ts = df['t_dat'].max()

# autox

In [6]:
articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', dtype={'article_id': str})
articles['sub_path'] = articles['article_id'].apply(lambda x: str(x)[:3])
articles = articles[["article_id", "sub_path", "detail_desc"]]

In [7]:
fe_img = fe_ima2vec(articles, 
                    img_path='../input/h-and-m-personalized-fashion-recommendations/images', 
                    img_col='article_id', 
                    filename_extension='jpg', 
                    sub_path = 'sub_path'
                   )

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

100%|██████████| 105542/105542 [2:59:05<00:00,  9.82it/s]


In [8]:
fe_img.index = articles.article_id
fe_img

,img_vec_0,img_vec_1,img_vec_2,img_vec_3,img_vec_4,img_vec_5,img_vec_6,img_vec_7,img_vec_8,img_vec_9,...,img_vec_502,img_vec_503,img_vec_504,img_vec_505,img_vec_506,img_vec_507,img_vec_508,img_vec_509,img_vec_510,img_vec_511
article_id,,,,,,,,,,,,,,,,,,,,,
0108775015,0.596330,0.252856,0.036450,1.520007,0.009170,1.393200,0.208283,2.201642,1.471440,0.135947,...,0.053520,0.093746,1.308659,0.995860,0.412460,0.898547,3.039079,1.296579,0.565731,2.159936
0108775044,0.567576,0.056198,0.003452,1.297433,0.019526,0.392277,0.044984,0.287056,1.966600,0.398177,...,0.094078,0.024092,1.721616,1.509363,1.684184,1.246171,0.575354,2.457583,0.678611,2.094800
0108775051,0.559030,0.167226,0.007577,0.651189,0.171190,0.763348,0.136167,0.919607,3.600429,0.561431,...,0.607297,0.111342,1.370912,1.734242,0.588848,0.932136,1.453882,1.726224,1.009761,1.356485
0110065001,0.321822,0.567115,1.510762,0.083512,0.572546,0.167765,0.249253,2.581934,0.519583,0.077566,...,0.572874,0.303835,0.670705,0.801034,0.162809,0.030235,1.548385,0.464793,0.254607,1.228232
0110065002,0.549902,0.097874,3.015078,1.753632,1.455938,0.576145,0.000000,0.723281,0.580283,0.462800,...,0.438721,0.498632,0.628150,1.234082,0.386859,0.091233,1.095976,3.126944,0.011339,0.845549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0953450001,2.714502,0.502266,3.173023,0.686610,0.404176,0.414200,0.013197,0.606686,1.399106,0.498086,...,0.903127,0.023044,0.197381,0.580682,0.064056,0.015875,0.994937,0.190669,0.494108,0.485490
0953763001,0.866329,0.052277,0.182056,1.672454,0.022855,2.258851,0.294014,0.865332,2.070608,0.509895,...,0.087527,0.221532,2.239783,0.927056,0.000000,1.157421,2.421481,1.601424,0.389788,0.291199
0956217002,0.452945,1.781943,0.116404,0.505772,0.022657,3.610852,0.085032,0.361071,3.332064,0.031930,...,0.164774,0.048484,1.460418,0.280183,0.107915,0.355932,0.764602,1.271283,0.526566,0.039019


In [9]:
fe_img.to_csv('autox_features.csv', index=None)

# als model

In [10]:
# for validation
tmp = df[df.week_no < test_week].reset_index(drop=True).copy()
    
# als model
all_users = tmp["customer_id"].unique().tolist()
all_items = tmp["article_id"].unique().tolist()

user_ids = dict(list(enumerate(all_users)))
item_ids = dict(list(enumerate(all_items)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {u: iidx for iidx, u in item_ids.items()}

tmp["customer_id"] = tmp["customer_id"].map(user_map)
tmp["article_id"] = tmp["article_id"].map(item_map)

row = tmp["customer_id"].values
col = tmp["article_id"].values

data = np.ones(tmp.shape[0])
coo = coo_matrix((data, (row, col)), shape=(len(all_users), len(all_items)))

n_factors = 5
als_model = implicit.als.AlternatingLeastSquares(factors=n_factors, iterations = 5, regularization=0.1, random_state=2240)
als_model.fit(coo)
item_factor_columns = ["article_id_"+str(i) for i in range(n_factors)]

del row, col, tmp, data, coo, user_ids, item_ids

with open('als_model_val.pickle', mode='wb') as f:
    pickle.dump(als_model, f)
    
with open('user_map_val.pickle', mode='wb') as f:
    pickle.dump(user_map, f)
    
with open('item_map_val.pickle', mode='wb') as f:
    pickle.dump(item_map, f)

WARNING -> OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading


  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
# for full train
tmp = df.copy()
    
# als model
all_users = tmp["customer_id"].unique().tolist()
all_items = tmp["article_id"].unique().tolist()

user_ids = dict(list(enumerate(all_users)))
item_ids = dict(list(enumerate(all_items)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {u: iidx for iidx, u in item_ids.items()}

tmp["customer_id"] = tmp["customer_id"].map(user_map)
tmp["article_id"] = tmp["article_id"].map(item_map)

row = tmp["customer_id"].values
col = tmp["article_id"].values

data = np.ones(tmp.shape[0])
coo = coo_matrix((data, (row, col)), shape=(len(all_users), len(all_items)))

n_factors = 5
als_model = implicit.als.AlternatingLeastSquares(factors=n_factors, iterations = 5, regularization=0.1, random_state=2240)
als_model.fit(coo)
item_factor_columns = ["article_id_"+str(i) for i in range(n_factors)]

del row, col, tmp, data, coo, user_ids, item_ids

with open('als_model_full.pickle', mode='wb') as f:
    pickle.dump(als_model, f)
    
with open('user_map_full.pickle', mode='wb') as f:
    pickle.dump(user_map, f)
    
with open('item_map_full.pickle', mode='wb') as f:
    pickle.dump(item_map, f)

  0%|          | 0/5 [00:00<?, ?it/s]

# first appperance week of article_id

In [12]:
article_week_range = df.groupby("article_id").week_no.agg(["min", "max"]).reset_index()
article_week_range.to_csv('article_week_range.csv', index=None)

# features by similaripy

In [13]:
# normalize matrix with bm25
#coo_bm = similaripy.normalization.bm25(coo)

# train the model with 50 knn per item 
#model = similaripy.cosine(coo_bm.T, k=50)

# recommend 100 items to users 1, 14 and 8 filtering the items already seen by each users
#user_recommendations = sim.dot_product(coo_bm, model.T, k=100, target_rows=[1,14,8], filter_cols=coo_bm)

# features by abstraction block

In [14]:
df = df[df.week_no > df.week_no.max() - 50].reset_index(drop=True)

In [15]:
article_columns = ["garment_group_no",
                "product_code", "product_type_no", 
                "graphical_appearance_no", "colour_group_code", "perceived_colour_value_id",
                "perceived_colour_master_id", "department_no", "index_group_no", 
                "section_no"]

df = pd.merge(df, article_df[["article_id"] + article_columns], on='article_id', how='left')

In [16]:
# customerごとに何か購入したweek
customer2weeks = df.groupby('customer_id')['week_no'].unique()

# customerごとに学習データの特徴量となるweekとラベルとなるweekのペアを作る。
customer2weekpairs = {}

for c_id, weeks in customer2weeks.items():
    customer2weekpairs[c_id] = {}
    for i in range(weeks.shape[0]-1):
        customer2weekpairs[c_id][weeks[i]] = weeks[i+1]
    customer2weekpairs[c_id][weeks[-1]] = test_week

In [17]:
def join(df):
    x = [str(e) for e in list(df)]
    return " ".join(x)

def label_prev_week(input_df):
    weeks = []
    for i, (c_id, week) in enumerate(zip(input_df['customer_id'], input_df['week_no'])):
        weeks.append(customer2weekpairs[c_id][week])
    input_df.week_no=weeks
    return input_df

In [18]:
class AbstractBaseBlock:
    def fit(self, input_df, y=None):
        return self.transform(input_df)

    def transform(self, input_df):
        raise NotImplementedError()
        
class UserHistoryEveryBlock(AbstractBaseBlock):
    """ユーザーの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['customer_id', 'week_no'])[self.item].agg(["sum", "std", "max", "min"]).add_prefix("u_prev_"+self.item+"_").reset_index()
        out = label_prev_week(out)
        return self.transform(out)

    def transform(self, input_df):
        return input_df
    
class UserHistoryVectorEveryBlock(AbstractBaseBlock):
    """ユーザーの文字履歴を週間で集計しベクトル化した特徴量を付与する block 累積ではない"""
    def __init__(self, item, n_components = 5):
        self.item = item
        self.n_components = n_components
        
    def fit(self, input_df, y=None):
        docs = input_df.groupby(["customer_id", "week_no"])[self.item].apply(join)
        max_features = int(input_df[self.item].nunique() * 0.8)
        tv = TfidfVectorizer(max_features=max_features)
        X = tv.fit_transform(docs)
        
        svd = TruncatedSVD(n_components=self.n_components, random_state=0)
        X = svd.fit_transform(X)
        result = pd.DataFrame(X, columns=[f"u_{self.item}_{i}" for i in range(self.n_components)])
        result.index = docs.index
        result = result.reset_index()
        result = label_prev_week(result)
        return self.transform(result)
  
    def transform(self, input_df):
        return input_df  

class ItemHistoryEveryBlock(AbstractBaseBlock):
    """アイテムの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['week_no'])[self.item].value_counts()
        out = out.to_frame("sale_count").reset_index()
        return self.transform(out)

    def transform(self, input_df):
        return input_df
    
class ItemHistorySumBlock(AbstractBaseBlock):
    """アイテムの数値履歴を週間で集計した特徴量を付与する block 累積ではない"""
    def __init__(self, item):
        self.item = item

    def fit(self, input_df, y=None):
        out = input_df \
        .groupby(['week_no', self.item])['price'].agg(["sum"]).add_prefix("sale_").reset_index()
        return self.transform(out)

    def transform(self, input_df):
        return input_df

In [19]:
# user_features
feature_blocks = [
#    UserHistoryEveryBlock("price"),
    UserHistoryVectorEveryBlock("section_no"),
    UserHistoryVectorEveryBlock("garment_group_no"),
]

for i, block in enumerate(feature_blocks):
    if i == 0:
        user_features_df = block.fit(df)
    else:
        out_i = block.fit(df)
        user_features_df = pd.merge(user_features_df, out_i, on = ["customer_id", "week_no"], how="left")
        
user_features_df.to_csv("user_features.csv", index=False)
del user_features_df

In [20]:
# item_features
article_count = ItemHistoryEveryBlock("article_id").fit(df)
article_count.to_csv("article_count.csv", index=False)

article_sale = ItemHistorySumBlock("article_id").fit(df)
article_sale.to_csv("article_sale.csv", index=False)